In [ ]:
pip install BeautifulSoup4

In [ ]:
pip install selenium

In [ ]:
pip install pandas_datareader

In [ ]:
pip install pandas

In [ ]:
pip install webdriver_manager

In [ ]:
!apt install chromium-chromedriver

In [ ]:
import os 
# declare a directory name 
dir_name= os.getcwd()+'/sentiment-data/'

In [ ]:


import pandas as pd

hkex_files=os.path.join(dir_name,'stock_ticker_datasets/hkex.csv')

hkex=pd.read_csv(hkex_files) 
# hkex[hkex.iloc['Category'] == 'Equity Securities']

hkex=hkex.loc[hkex['Category'] == 'Equity']

hkex['Ticker']=hkex['Ticker'].astype(str)
hkex_input=hkex['Ticker']

n = 400  #chunk row size
hkex_df = [hkex_input[i:i+n] for i in range(0,len(hkex_input),n)]


In [ ]:
# Collect all stocks using selenium 

from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen
import csv
import datetime, time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument('no-sandbox')  
chrome_options.add_argument('-disable-dev-shm-usage')  

# target_time='2020-01-01'
# target_time=datetime.datetime.strptime(target_time, '%Y-%m-%d')

# get news from aastock.com
def get_news_aastock(ticker,postfix_url,newstype):
#     driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
    driver.implicitly_wait(20)
#     WebDriverWait wait = new WebDriverWait(driver, 10);
    prefix_url='http://www.aastocks.com/en/stocks/analysis/stock-aafn/'
#     postfix_url='/0/research-report/1'
    try:
        SCROLL_PAUSE_TIME = 2
        fill_ticker=ticker.zfill(5)
        url=prefix_url+fill_ticker+postfix_url
        driver.get(url)
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

     
        html=BeautifulSoup(driver.page_source, 'lxml')
      
        dates=html.findAll("div", {"class": "newstime4"})
        
        news=html.findAll("div", {"class": "newshead4"})
        
        idx=0
        path=os.path.join(dir_name,'data-news/data-aastock-equities/'+'data-'+fill_ticker+'-aastock.csv')
        if (len(dates)>0):
            with open(path,'a') as f:
                writer = csv.writer(f)
                for i in dates:
                    
                    if "/" in str(i.get_text()):
                        print(idx)
                        
                        date=str(i.get_text())
                        if "Release Time" in date:
                            date=date[13:23]
                        elif (date[0]==" "):
                            date=str(date[1:11])
                        else:
                            date=str(date[0:10])
#                         print(date)
                        text=news[idx].get_text()

                        date_time_obj = datetime.datetime.strptime(date, '%Y/%m/%d')
#                         if(date_time_obj>=target_time):
                        date_time=date_time_obj.strftime('%Y-%m-%d')
#                             print([date_time,text])
#                         print(([date_time,text,ticker,'news']))
                        writer.writerow([date_time,text,ticker,newstype])

                        idx+=1
    except Exception as e:
        print(e)
        pass
    driver.quit()
    


In [ ]:
# Collect all stocks using selenium within a specific day

from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen
import csv
import datetime, time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument('no-sandbox')  
chrome_options.add_argument('-disable-dev-shm-usage')  




# target_time='2020-01-01'
# target_time=datetime.datetime.strptime(target_time, '%Y-%m-%d')

# get news from aastock.com
def get_news_aastock_time(ticker,postfix_url,newstype,days):
#     driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
    driver.implicitly_wait(20)
#     WebDriverWait wait = new WebDriverWait(driver, 10);
    prefix_url='http://www.aastocks.com/en/stocks/analysis/stock-aafn/'
#     postfix_url='/0/research-report/1'
    try:
        SCROLL_PAUSE_TIME = 2
        fill_ticker=ticker.zfill(5)
        url=prefix_url+fill_ticker+postfix_url
        driver.get(url)
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
     
        html=BeautifulSoup(driver.page_source, 'lxml')
      
        dates=html.findAll("div", {"class": "newstime4"})
        
        news=html.findAll("div", {"class": "newshead4"})
        
        idx=0
        path=os.path.join(dir_name,'data-news/data-aastock-equities/'+'data-'+fill_ticker+'-aastock.csv')
        if (len(dates)>0):
            with open(path,'a') as f:
                writer = csv.writer(f)
                for i in dates:
                    
                    if "/" in str(i.get_text()):
                        print(idx)
                        
                        date=str(i.get_text())
                        if "Release Time" in date:
                            date=date[13:23]
                        elif (date[0]==" "):
                            date=str(date[1:11])
                        else:
                            date=str(date[0:10])
#                         print(date)
                        text=news[idx].get_text()

                        date_time_obj = datetime.datetime.strptime(date, '%Y/%m/%d')
                        if(datetime.datetime.now()-date_time_obj).days<=days:
                            date_time=date_time_obj.strftime('%Y-%m-%d')
                            print([date_time,text])
                            print(([date_time,text,ticker,'news']))
                            writer.writerow([date_time,text,ticker,newstype])

                        idx+=1
    except Exception as e:
        print(e)
        pass
    driver.quit()
    

In [ ]:
# collect all the news in aastock for each ticker
def collect_news (ticker):
    
    news_report_postfix_url='/0/research-report'
    news_result_postfix_url='/0/result-announcement'
    news_daily_postfix_url='/0/hk-stock-news'
    news_indus_postfix_url='/0/industry-news'
    
    get_news_aastock(ticker,news_daily_postfix_url,'news-daily')
    get_news_aastock(ticker,news_report_postfix_url,'news-report')
    get_news_aastock(ticker,news_result_postfix_url,'news-result')
    get_news_aastock(ticker,news_indus_postfix_url,'news-indus')

for tickers in hkex_df:
     for ticker in tickers:
                try:
                        print (ticker)
                        collect_news(ticker)
                except Exception as e:
                    print(e)
                    pass

In [ ]:
# collect all the news in aastock for each ticker within a speicific time
def collect_news_time (ticker,days):
    
    news_report_postfix_url='/0/research-report'
    news_result_postfix_url='/0/result-announcement'
    news_daily_postfix_url='/0/hk-stock-news'
    news_indus_postfix_url='/0/industry-news'
    
    get_news_aastock_time(ticker,news_daily_postfix_url,'news-daily',days)
    get_news_aastock_time(ticker,news_report_postfix_url,'news-report',days)
    get_news_aastock_time(ticker,news_result_postfix_url,'news-result',days)
    get_news_aastock_time(ticker,news_indus_postfix_url,'news-indus',days)

for tickers in hkex_df:
     for ticker in tickers:
                try:
                        print (ticker)
                        collect_news_time(ticker,45)
                except Exception as e:
                    print(e)
                    pass

In [ ]:
import os 
import datetime, time
import pandas as pd

#agg function for all equities in hkex
def agg_news(hkex_df,df_hkex):
    
    for ticker in df_hkex:
        try:
            print(ticker)
            news_path=os.path.join(dir_name,'data-news/data-aastock-equities/'+'data-'+ticker.zfill(5)+'-aastock.csv')
            
            df = pd.read_csv(news_path,names=['dates','news','ticker','newstype'])

            hkex_df=hkex_df.append(df)  
            print(hkex_df)

        except Exception as e:
            print(e)
            pass
        
    return hkex_df


In [ ]:
import csv
# get agg news for all equities in hkex
df_hkex= pd.DataFrame()
df=agg_news(df_hkex,hkex_input)
print(df)
   